In [2]:
from sklearn.datasets import load_iris
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import mglearn
import scipy as sp
import sklearn
from sklearn.model_selection import train_test_split
%matplotlib inline

/opt/anaconda3/envs/Titanic/lib/python3.7/site-packages/sklearn/externals/six.py:31: FutureWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", FutureWarning)
/opt/anaconda3/envs/Titanic/lib/python3.7/site-packages/sklearn/externals/joblib/__init__.py:15: FutureWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=FutureWarning)


In [3]:
#データの読み込み
df_train = pd.read_csv('train.csv')
#読み込んだデータの確認
df_train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [4]:
#各行・列ごとに欠損値を一つでも含むか確認
df_train.isnull().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

In [6]:
#欠損値処理
df_train['Fare'] = df_train['Fare'].fillna(df_train['Fare'].median())
df_train['Age'] = df_train['Age'].fillna(df_train['Age'].median())
df_train['Embarked'] = df_train['Embarked'].fillna('S')

In [8]:
#カテゴリ変数を変換
df_train['Sex'] = df_train['Sex'].apply(lambda x:1 if x=='male' else 0)
df_train['Embarked'] = df_train['Embarked'].map({'S':0,'C':1,'Q':2}).astype(int)

In [9]:
#不要なcolumnを削除
df_train = df_train.drop(['Cabin','Name','PassengerId','Ticket'],axis=1)

In [10]:
#学習データとテストデータに分割
train_X = df_train.drop('Survived',axis=1)
train_y = df_train.Survived
train_X,test_X,train_y,test_y = train_test_split(train_X,train_y,test_size=0.3,random_state=0)

In [13]:
from sklearn.ensemble import RandomForestClassifier
model1 = RandomForestClassifier(n_estimators=100,max_features=3,max_depth=10)
model1.fit(train_X,train_y)
print(model1.score(train_X,train_y))
print(model1.score(test_X,test_y))

0.9486356340288925
0.835820895522388


In [14]:
#テストデータにも同じことをする
df_test = pd.read_csv('test.csv')
df_test.head()

passsengerid = df_test['PassengerId']
df_test.isnull().sum()
df_test['Fare'] = df_test['Fare'].fillna(df_test['Fare'].median())
df_test['Age'] = df_test['Age'].fillna(df_test['Age'].median())
df_test['Embarked'] = df_test['Embarked'].fillna('S')

#カテゴリ変数の変換
df_test['Sex'] = df_test['Sex'].apply(lambda x: 1 if x == 'male' else 0)
df_test['Embarked'] = df_test['Embarked'].map( {'S': 0 , 'C':1 , 'Q':2}).astype(int)

#不要なcolumnを削除
df_test= df_test.drop(['Cabin','Name','Ticket','PassengerId'],axis =1)

In [16]:
prediction = model1.predict(df_test)

In [17]:
submission = pd.DataFrame({'PassengerId':passsengerid, 'Survived':prediction})
submission.to_csv('submission.csv' , index = False)

In [19]:
#パラメータ変えてみる
model2 = RandomForestClassifier(n_estimators=100,max_features=7,max_depth=10)
model2.fit(train_X,train_y)
print(model2.score(train_X,train_y))
print(model2.score(test_X,test_y))

0.956661316211878
0.8246268656716418
